In [ ]:
import drjit as dr
import mitsuba as mi
import numpy as np
from PIL import Image
mi.set_variant('llvm_ad_rgb')

In [ ]:
meters_per_sample = 30
num_samples = 60*60

In [ ]:
def create_terrain_mesh(num_samples, meters_per_sample, DSM):
    # Generate UV coordinates
    U, V = dr.meshgrid(
        dr.linspace(mi.Float, 0, 1, num_samples),
        dr.linspace(mi.Float, 0, 1, num_samples),
        indexing='ij'
    )
    texcoords = mi.Vector2f(U, V)
    
    # Generate vertex coordinates
    X = num_samples * meters_per_sample * U
    Y = num_samples * meters_per_sample * V
    
    DSM = np.rot90(DSM)
    vertices = mi.Vector3f(X, Y, DSM.ravel())
    
    # Create two triangles per grid cell
    faces_x, faces_y, faces_z = [], [], []
    for i in range(num_samples - 1):
        for j in range(num_samples - 1):
            v00 = i * num_samples + j
            v01 = v00 + 1
            v10 = (i + 1) * num_samples + j
            v11 = v10 + 1
            
            faces_x.extend([v00, v01])
            faces_y.extend([v10, v10])
            faces_z.extend([v01, v11])

    # Assemble face buffer
    faces = mi.Vector3u(faces_x, faces_y, faces_z)

    # Instantiate the mesh object
    mesh = mi.Mesh("terrain-mesh", num_samples * num_samples, len(faces_x), has_vertex_texcoords=True)

    # Set its buffers
    mesh_params = mi.traverse(mesh)
    mesh_params['vertex_positions'] = dr.ravel(vertices)
    mesh_params['vertex_texcoords'] = dr.ravel(texcoords)
    mesh_params['faces'] = dr.ravel(faces)
    mesh_params.update()

    return mesh

In [ ]:
DSM = np.array(Image.open('N046E007/ALPSMLC30_N046E007_DSM.tif'))
mesh = create_terrain_mesh(num_samples, meters_per_sample,DSM)
mesh.write_ply('DSM.ply')

In [ ]:
# Looking at the receiving plane, not looking through the lens
'''
sensor_to_world = mi.ScalarTransform4f.look_at(
    target=[1800*30, 1800*30, 0],
    origin=[1800*30, 1500*30, 40_000],
    up=[0, 0, 1]
)
'''
sensor_to_world = mi.ScalarTransform4f.translate([1800*30, 1800*30, 40_000]).rotate(axis=[1, 0, 0], angle=180).scale([1800*30, 1800*30, 1])

sensor = {
    'type': 'orthographic',
    'near_clip': 1,
    'far_clip': 250_000,
    'to_world': sensor_to_world,
    'sampler': {'type': 'ldsampler'},
    'film': {
        'type': 'hdrfilm',
        'width': 1_000,
        'height': 1_000,
        'rfilter': { 'type': 'gaussian' },
        'sample_border': True
    },
}

scene = mi.load_dict({
    'type': 'scene',
    'sensor': sensor,
    'integrator': {'type':'path'},
    'DSM': {
        'type': 'ply',
        'id': 'DSM',
        'filename': 'DSM.ply',
        'bsdf': {
            'type': 'diffuse',
            'reflectance': { 'type': 'rgb', 'value': (0.5,0.5,0.5)}
        },
    },
    'light': {
           'type':'envmap',
            'filename': 'abandoned_tank_farm_04.exr',
            'to_world': mi.ScalarTransform4f.rotate(axis=(1, 0, 0), angle=90),
    },
})

In [ ]:
params = mi.traverse(scene)

In [ ]:
image = mi.render(scene, params, spp = 1_024)
bitmap = mi.util.convert_to_bitmap(image)

In [ ]:
bitmap.write('render.png')